# Bibliotecas

In [1]:
import pandas as pd
import random as rd 
import time as tm

# Conjunto de Dados

In [2]:
csv_vendas_201910 = 'base/2019-Oct.csv'

# Amostra dos dados (Random)

In [4]:
total_linhas = sum(1 for linha in open(csv_vendas_201910))
total_linhas_amostra = total_linhas//10

linhas_sortidas = sorted(rd.sample(range(1,total_linhas+1), total_linhas-total_linhas_amostra))

df_vendas_201910_amostra = pd.read_csv(csv_vendas_201910, skiprows=linhas_sortidas)

In [5]:
df_vendas_201910_amostra.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
1,2019-10-01 00:00:17 UTC,view,23100006,2053013561638126333,NaN,NaN,357.79,513642368,17566c27-0a8f-4506-9f30-c6a2ccbf583b
2,2019-10-01 00:00:28 UTC,view,26200591,2053013563693335403,NaN,NaN,203.35,548449430,99617d1c-1b5a-42f8-99f1-42ad83a6155f
3,2019-10-01 00:00:37 UTC,view,1701111,2053013553031414015,computers.peripherals.monitor,acer,514.79,547028884,3ea7c620-a8d7-45c5-9ced-2e9874e2f549
4,2019-10-01 00:00:44 UTC,view,31501163,2053013558031024687,NaN,luminarc,128.45,550978835,6280d577-25c8-4147-99a7-abc6048498d6


# Dados completos (Read CSV)

In [6]:
inicio = tm.time()
df_vendas_201910 = pd.read_csv(csv_vendas_201910)
fim = tm.time()
print("Duração:",round((fim-2),2), "segundos")
print("Tamanho:",round(df_vendas_201910.memory_usage(deep=True).sum() / 1024 ** 2,2), 'MB')

Duração: 1613254314.1 segundos
Tamanho: 15743.62 MB


Dados completos (Read/ CSV) - Análise de Memória

In [7]:
df_vendas_201910.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4 GB


In [8]:
for tipo_dados in ['float','int64','object']:
    selecao_tipos_dados = df_vendas_201910.select_dtypes(include=[tipo_dados])
    total_usado_b = selecao_tipos_dados.memory_usage(deep=True).sum()
    total_usado_mb = round(total_usado_b / 1024 ** 2,2)
    print("Total de Memória utilizada por {0}: {1} MB".format(tipo_dados,total_usado_mb))

Total de Memória utilizada por float: 323.86 MB
Total de Memória utilizada por int64: 971.58 MB
Total de Memória utilizada por object: 14448.18 MB


# Dados completos em blocos (Chunking)

In [10]:
inicio = tm.time()
chunks = []

for chunk in pd.read_csv(csv_vendas_201910, chunksize=1000000):
    chunks.append(chunk)
df_vendas_201910_chunk = pd.concat(chunks)

fim = tm.time()
print("Duração:",round((fim-inicio),2), "segundos")
print("Tamanho:",round(df_vendas_201910_chunk.memory_usage(deep=True).sum() / 1024 ** 2,2), 'MB')
memoria_df_vendas_201910_chunk = round(df_vendas_201910_chunk.memory_usage(deep=True).sum() / 1024 ** 2,2)

Duração: 188.27 segundos
Tamanho: 15743.62 MB


# Dados completos em blocos (Chunking) - Análise de Memória

In [11]:
df_vendas_201910_chunk.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42448764 entries, 0 to 42448763
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.4 GB


In [12]:
for tipo_dados in ['float','int64','object']:
    selecao_tipos_dados = df_vendas_201910_chunk.select_dtypes(include=[tipo_dados])
    total_usado_b = selecao_tipos_dados.memory_usage(deep=True).sum()
    total_usado_mb = round(total_usado_b / 1024 ** 2,2)
    print("Total de Memória utilizada por {0}: {1} MB".format(tipo_dados,total_usado_mb))

Total de Memória utilizada por float: 323.86 MB
Total de Memória utilizada por int64: 971.58 MB
Total de Memória utilizada por object: 14448.18 MB


# Dados completos (dtype)

In [15]:
lista_campos_tipo_objeto = df_vendas_201910_chunk.select_dtypes('object').columns

df_vendas_201910_chunk[lista_campos_tipo_objeto] = df_vendas_201910_chunk[lista_campos_tipo_objeto].astype('category')

print("Tamanho:",round(df_vendas_201910_chunk.memory_usage(deep=True).sum() / 1024 ** 2,2), 'MB')
print("Redução de", ((memoria_df_vendas_201910_chunk - (df_vendas_201910_chunk.memory_usage(deep=True).sum() / 1024 ** 2)) / memoria_df_vendas_201910_chunk)*100,'%')


Tamanho: 3123.98 MB
Redução de 80.15716049513651 %


# Dask

In [16]:
!pip install dask[dataframe]

In [19]:
from dask import dataframe as dd

inicio = tm.time()
dd_vendas_2019_dask = dd.read_csv(csv_vendas_201910)
df_vendas_201910_dask = dd_vendas_2019_dask.compute(num_workers=8)
fim = tm.time()
print("Duração:",round((fim-inicio),2), "segundos")
df_vendas_201910_dask.info(memory_usage='deep')

Duração: 108.23 segundos
<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 236342
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.7 GB


In [20]:
inicio = tm.time()
dd_vendas_2019_dask = dd.read_csv(csv_vendas_201910,blocksize=100 * 1024 * 1024)
df_vendas_201910_dask = dd_vendas_2019_dask.compute(num_workers=8)
fim = tm.time()
print("Duração:",round((fim-inicio),2), "segundos")
df_vendas_201910_dask.info(memory_usage='deep')

Duração: 123.07 segundos
<class 'pandas.core.frame.DataFrame'>
Int64Index: 42448764 entries, 0 to 47285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 15.7 GB
